In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/elleven/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoderbase-1b", use_fast=False)

In [3]:
model = AutoModelForCausalLM.from_pretrained("bigcode/starcoderbase-1b").cuda()

In [148]:
PROMPT = """
-- Task
-- We are given two strings s and c, you have to deleted all the characters in s 
-- that are equal to any character in c
-- then check if the result string is palindrome.
-- A string is called palindrome if it reads the same backward as forward.
-- You should return a tuple containing the result string and True/False for the check.
--
-- Example
-- For s = "abcde", c = "ae", the result should be ('bcd',False)
-- For s = "abcdef", c = "b" the result should be ('acdef',False)
-- For s = "abcdedcba", c = "ab", the result should be ('cdedc',True)
local function reverse_delete(s, c)
"""

In [149]:
def find_end_tok_i(tokenizer, enc, stop_seqs=["\nend", "\n--"]):
    def stop_in_enc(enc):
        dec = tokenizer.decode(enc)
        for stop in stop_seqs:
            if stop in dec:
                return True

        return False
        
    i = 0
    
    while i < len(enc) - 1 and not stop_in_enc(enc[:i]):
        i += 1

    return i
    
    
toks = tokenizer.encode(PROMPT, return_tensors="pt").cuda()

In [150]:
out = model.generate(toks, do_sample=True, max_new_tokens=150, temperature=0.2, top_p=0.95, stopping_criteria=stopping_criteria)
end_tok = find_end_tok_i(tokenizer, out[0][len(toks[0])-1:])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [152]:
print(tokenizer.decode(out[0]))


-- Task
-- We are given two strings s and c, you have to deleted all the characters in s 
-- that are equal to any character in c
-- then check if the result string is palindrome.
-- A string is called palindrome if it reads the same backward as forward.
-- You should return a tuple containing the result string and True/False for the check.
--
-- Example
-- For s = "abcde", c = "ae", the result should be ('bcd',False)
-- For s = "abcdef", c = "b" the result should be ('acdef',False)
-- For s = "abcdedcba", c = "ab", the result should be ('cdedc',True)
local function reverse_delete(s, c)
	local result = ""
	for i = 1, #s do
		if s:sub(i, i) ~= c:sub(1, 1) then
			result = result.. s:sub(i, i)
		end
	end
	return result, true
end

-- Solution
-- Time: O(n)
-- Space: O(1)
function reverse_delete(s, c)
	local result = ""
	for i = 1, #s do
		if s:sub(i, i) ~= c:sub(1, 1) then
			result = result.. s:sub(i, i)
	


In [172]:
enc = model(out, output_attentions=True)
attns = enc["attentions"]
# quite a deep tensor...
layer_i = 0
batch_i = 0 # we only have one prompt
attn_head_i = 0
print(end_tok)
attns[layer_i][batch_i][attn_head_i][end_tok+30]

# sum all layers
s_layer = attns[0][batch_i]
for i in range(1, len(attns[layer_i][batch_i])):
    s_layer += attns[i][batch_i]

# sum all heads
summed = s_layer[0]
for i in range(1, len(s_layer)):
    summed += s_layer[i]

67


tensor([256.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   

In [180]:
summed[7]

tensor([191.0103,   5.0398,   7.3868,   9.1133,   8.0260,  11.1349,  14.6481,
          9.6409,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.00